# Notes


2015-12-22 

- tanh units
    - (dropout generally 0.5)
    - dense hingeloss ok
    - identity // sigmoid hingeloss doesn't seem to work (inner scale 1.05)
    - softmax_binary_crossentropy doesnt' seem to work.
- relu units
    - inner scale 1.0

#####
 TODO 7/2015 :: Figure out why the .ipynb has to be in ~/keras 
 (and NOT ~/notebooks, and NOT ~/keras/keras, and NOT ~/keras/keras/notebooks)
 in order to run from keras.optimizers import HF
 UPDATE 12/2015 :: I think I over-wrote this code so it doesn't matter any more.
from keras.optimizers import SGD
from keras.optimizers import HF


#### PARAMETERS #########
 Hyperparameters to search
 - RNN hidden intialization.  Something around 1-1.1 is reasonable.
     -> implemented as rotation(identity * [1..1.1]), but vary rotation
     -> implemented

 - RNN input initialization.  Gaussian(0.1) ish?
     - input as init = normal(shape, scale=0.05)

 - Centering using mean-subtraction.  This is actually a signal, so be careful.
     - X_train = X_train - mean()
    for row in range(a.shape[0]):
        a[row,:,:] = a[row,:,:] - np.mean( a[row,:,:] )
    --> implemented ++

 TODO
 - validation holdout set.
 - softmax gives very bad values (-16 forever?)

# TO Try
- centering / not centering
- slightly greater than 1
- rotated identity
- start with easy trials, get harder later.


## RESULTS


#### tanh / dense hingeloss / identity / rmsprop / dropout 0.75 --> ok model
dense_input_dim      512
dense_output_dim     1
do_center_X          0
dropout              0.75
final_layer_type     'dense_hingeloss'
inner_activation     'tanh'
optimizer            'RMSprop'
regenerate_datasets  1
rmsprop_clipvalue    None
rmsprop_lr           0.001
rnn_1_hidden_dim     512
rnn_1_input_dim      2
rnn_init_scale       1 --> probably way too large
rnn_inner_init_type  'identity'
rnn_inner_scale      1.0
100000/100000 [==============================] - 281s - loss: 0.4130 - val_loss: 0.2956
Epoch 00006: val_loss improved from 0.46421 to 0.29556, saving model to /home/avaughan/Dropbox/rnn/checkpoint_2016-01-12_01-37-11.hdf5

##### tanh / dense hingeloss / identity / rmsprop / dropout 0.75 --> STUCK
-------------------  -----------------------------
dense_input_dim      512
dense_output_dim     1
do_center_X          1
dropout              0
final_layer_type     'softmax_binary_crossentropy'
inner_activation     'tanh'
optimizer            'RMSprop'
regenerate_datasets  1
rmsprop_clipvalue    None
rmsprop_lr           1e-06
rnn_1_hidden_dim     512
rnn_1_input_dim      2
rnn_init_scale       0.001
rnn_inner_init_type  'identity'
rnn_inner_scale      1.0
-------------------  -----------------------------


#### tanh / sigmoid hingeloss / identity / rmsprop / dropout 0 or 0.5 --> STUCK
dense_input_dim      512
dense_output_dim     1
do_center_X          1
dropout              0
final_layer_type     'sigmoid_hingeloss'
inner_activation     'tanh'
optimizer            'RMSprop'
regenerate_datasets  1
rmsprop_clipvalue    None
rmsprop_lr           1e-06
rnn_1_hidden_dim     512
rnn_1_input_dim      2
rnn_init_scale       0.001
rnn_inner_init_type  'identity'
rnn_inner_scale      1.0
-------------------  -------------------
Epoch 12/300
100000/100000 [==============================] - 333s - loss: 0.5014 - val_loss: 0.4912

### RELU / dense hingeloss / identity / rmsprop 1E-6 / dropout  0.5 --> WORKS but eventually throws a nan.  Later though!
Note, still have 2d sigmoid layer here.
Changing to sigmoid hingeloss throws a nan sooner - still saturating?
-------------------  -------------------
dense_input_dim      128
dense_output_dim     1
do_center_X          1
dropout              0.5
final_layer_type     'dense_hingeloss'
inner_activation     'relu'
optimizer            'RMSprop'
regenerate_datasets  0
rmsprop_clipvalue    None
rmsprop_lr           1E-6
rnn_1_hidden_dim     128
rnn_1_input_dim      2
rnn_init_scale       0.001
rnn_inner_init_type  'identity'
rnn_inner_scale      1.0
-------------------  -------------------

rnn_output_scale     0.0001 --> NAN on batch 119.

##
final_layer_type     'sigmoid_hingeloss' --> Batchwise loss was NaN in batch 2302!
##
rnn_output_scale     0.00001 --> NAN loss on batch 119/2000.  Not sure.  Keep bashing on this, it's close.

## RUNS
but is basically stuck... long asymptote at 0.4958 loss
-------------------  -------------------
dense_input_dim      128
dense_output_dim     1
do_center_X          1
dropout              0.5
final_layer_type     'sigmoid_hingeloss'
inner_activation     'relu'
optimizer            'RMSprop'
regenerate_datasets  0
rmsprop_clipvalue    None
rmsprop_lr           0.001
rnn_1_hidden_dim     128
rnn_1_input_dim      2
rnn_init_scale       0.001
rnn_inner_init_type  'identity'
rnn_inner_scale      1.0
rnn_output_scale     1e-05
-------------------  -------------------
This also has ... model.add(Dense( 2, init = lambda shape: normal(shape,scale=params.rnn_output_scale), activation='sigmoid'))
as the penultimate layer
--> doesn't work with dropout 0.9 --> NAN
--> 0.75 works with 128 batch size.
--> Tried SGD with learning rate 1e-3..1e-5 --> failes with nan.
--> Back to RMSprop, with inner scale up to 1.1.  Stops at 0.495 error again.


In [1]:
%matplotlib inline

########## IMPORTS AND UTILITY FUNCITONS / CLASSES ###########

from __future__ import division
from __future__ import print_function
import numpy as np
import scipy as sp
import theano as T
import pandas
import time, random, sys

from keras import backend
print('Backend is :: ',backend._BACKEND)

#from keras.datasets.data_utils import get_file

import matplotlib.pyplot as plt
#import matplotlib.cm as cm

from __future__ import print_function
from six.moves import range
from datetime import datetime
import string
from tabulate import tabulate

# My function for generating poisson datasets
import generate_poisson_dataset
reload(generate_poisson_dataset)

# Utility class stolen from 3.3
class SimpleNameSpace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    def __repr__(self):
        keys = sorted(self.__dict__)
        items = ("{}={!r}".format(k, self.__dict__[k]) for k in keys)
        return "{}({})".format(type(self).__name__, ", ".join(items))
    def __eq__(self, other):
        return self.__dict__ == other.__dict__
    
print('Done at ',datetime.now())



Couldn't import dot_parser, loading of dot files will not be possible.
Using Theano backend.
Backend is ::  theano
Done at  2016-01-12 23:29:54.704273


Using gpu device 0: GeForce GTX 980 (CNMeM is disabled)


In [2]:
# TODO 7/2015 :: Figure out why the .ipynb has to be in ~/keras 
# (and NOT ~/notebooks, and NOT ~/keras/keras, and NOT ~/keras/keras/notebooks)
# in order to run from keras.optimizers import HF
# UPDATE 12/2015 :: I think I over-wrote this code so it doesn't matter any more.
#from keras.optimizers import SGD
#from keras.optimizers import HF

In [28]:

####### Function to define model #########

def define_and_compile_model(params):
    
    # Print parameters
    params_str = str(params)
    print( 'Compiling with parameters as follows:')
    print( tabulate(string.split(p,'=') for p in string.split(params_str[9:-1],',')))
    
   
    ####### Initialize inner weights of RNN #####

    def normal_radius(shape,scale=1):
        normal_init = np.random.randn(*shape)
        radius_init = max(abs(sp.linalg.eigvals(normal_init)))
        return K.variable( normal_init / radius_init * scale )

    if params.rnn_inner_init_type == 'identity':
        rnn_inner_init = identity
    elif params.rnn_inner_init_type == 'normal_radius':
        rnn_inner_init = normal_radius
    elif params.rnn_inner_init_type == 'orthogonal':
        rnn_inner_init = orthogonal
    else:
        raise Exception('params.rnn_inner_init_type not found')

    ####### Initialize Final Layer #######

    if params.final_layer_type == 'dense_hingeloss':
        final_layer = Dense(1, init=lambda shape: normal(shape,scale=0.1))
        loss_fn = 'hinge'
        optimizer_choice_values = (-1,1)
    
    elif params.final_layer_type == 'softmax_binary_crossentropy':
        #final_layer = Dense(1,activation='softmax',init=lambda shape: normal(shape,scale=0.1))
        final_layer = Dense(1,activation='softmax',init='glorot_uniform')
        loss_fn = binary_crossentropy
        optimizer_choice_values = (0,1)
        
    elif params.final_layer_type == 'softmax_categorical_crossentropy':
        final_layer = Dense(2,activation='softmax')
        loss_fn = categorical_crossentropy
        optimizer_choice_values = (0,1)
    
    elif params.final_layer_type == 'sigmoid_hingeloss':
        final_layer = Dense(1,activation='sigmoid', init=lambda shape: normal(shape,scale=0.1))
        loss_fn = 'hinge'
        optimizer_choice_values = (0,1)
    
    else:
        raise Exception('params.final_layer_type not found :: ',params.final_layer_type)
        

    ####### Define loss and optimizer for model ######

    def two_afc_loss(y_true, y_pred):
        # Possible contributions to loss
        mean_pre_output  = T.mean(abs(y_pred[:-1]))
        not_rewarded = 1 - ( T.gt( y_true[-1] * y_pred[-1] , abs(y_true[-1]) ) ) # change 1 --> abs(y_true[-1])
        binary_choice_error = 1 - T.gt(abs(y_pred[-1]-y_true[-1]),0) # 0 if correct, 1 if incorrect
        abs_choice_error = abs(y_true[-1] - y_pred[-1])           # abs magnitude of error
        # Output loss :: we include a T.zeros_like to make sure that our objective is appropriately broadcast to the size of the original inputs.
        return abs_choice_error + T.zeros_like(y_true) # convert to a loss function, where 1 is a failed trial

    if params.optimizer == 'SGD':
        #optimizer = optimizers.SGD(lr=1e-4, decay=1e-5, momentum=0.99, nesterov=True)
        if params.sgd_clipvalue is None:
            optimizer = optimizers.SGD(lr=params.sgd_lr, decay=params.sgd_decay, momentum=params.sgd_momentum, nesterov=params.sgd_nesterov)
        else:
            optimizer = optimizers.SGD(lr=params.sgd_lr, decay=params.sgd_decay, momentum=params.sgd_momentum, nesterov=params.sgd_nesterov,clipvalue=params.sgd_clipvalue)
    elif params.optimizer == 'RMSprop':
        # Leave at default I guess?
        if params.rmsprop_clipvalue is None:
            optimizer = optimizers.RMSprop()
        else:
            optimizer = optimizers.RMSprop(clipvalue=params.rmsprop_clipvalue)
            
    else:
        raise Exception('params.optimizer not found :: ', params.optimizer)

    ####### Define model ######

    print('Building  model...')
    model = Sequential()
    model.add( recurrent.SimpleRNN( params.rnn_1_hidden_dim,
                                   input_shape      = (1001,2),
                                   activation       = params.inner_activation,
                                   return_sequences = False,
                                   init             = lambda shape: normal(shape,scale=params.rnn_init_scale),
                                   inner_init       = lambda shape: rnn_inner_init(shape,scale=params.rnn_inner_scale),
                                   ))
    model.add(Dropout(params.dropout))
    model.add(Dense( 2,
                    init = lambda shape: normal(shape,scale=params.rnn_output_scale),
                    activation='sigmoid'))
    model.add( final_layer )

    ######## Compile ########
    
    print('Compiling')
    t_start = time.time()
    model.compile(loss=loss_fn, optimizer=optimizer)
    print('...done compiling in %.1f seconds' %(time.time()-t_start),datetime.now())
    return model

print('\nDone with cell at',datetime.now())



Done with cell at 2016-01-19 20:06:38.417384


In [14]:
###########################################
######### SPECIFY PARAMETERS ##############
###########################################

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import recurrent
import keras.optimizers as optimizers
import keras.callbacks as callbacks
from keras.objectives import hinge, binary_crossentropy, categorical_crossentropy
from keras.initializations import normal,identity,orthogonal
from keras import backend as K
reload(callbacks)
reload(generate_poisson_dataset)

############## GENERATE DATA #############

n_trials_train = 100000
n_trials_test  = 10000
stim_duration  = 0.5
wait_time      = 0
sampling_freq  = 1000
sum_rate       = 200
n_samples = (stim_duration+wait_time)*sampling_freq + 1
regenerate_datasets = True

def generate_dataset():

    t_start = time.time()
    click_bias_train  = np.random.uniform( 0, 1, (n_trials_train))
    click_bias_test   = np.random.uniform( 0, 1, (n_trials_test) )
    (X_train,Y_train) = generate_poisson_dataset.generate_click_dataset( 
        n_trials_train,  click_bias_train, n_samples, 
        sum_rate=sum_rate, stim_duration=stim_duration, 
        wait_time=wait_time, sampling_freq=sampling_freq, 
        centered=False,choice_values=choice_values)
    (X_test,Y_test)   = generate_poisson_dataset.generate_click_dataset( 
        n_trials_test,  click_bias_test, n_samples, 
        sum_rate=sum_rate, stim_duration=stim_duration,  
        wait_time=wait_time,  sampling_freq=sampling_freq, 
        centered=False,choice_values=choice_values)
    
    print('Datasets generated in %.1f seconds' %( time.time() - t_start))

    if params.do_center_X:
        print('Centering datasets...')
        for row in range(X_train.shape[0]):
            X_train[row,:,:] = X_train[row,:,:] - np.mean( X_train[row,:,:] )
        for row in range(X_test.shape[0]):
            X_test[row,:,:] = X_test[row,:,:] - np.mean( X_test[row,:,:] )
    else:
        print('Not centering datasets.')
    return (X_train,Y_train,X_test,Y_test)
        
print('\nDone with cell at',datetime.now())


Done with cell at 2016-01-19 19:02:37.962221


In [54]:
######################################################
######### GENERATE DATSET AND RUN MODEL ##############
######################################################

####### Compile the actual model #########
params = SimpleNameSpace()

params.rnn_1_input_dim               = 2
params.rnn_1_hidden_dim              = 256 # = number of hidden units.

params.rnn_init_scale                = 1 # Normal, scaled by sigma (std).  ~0.001ish might be reasonable.
params.rnn_inner_init_type           = 'identity' 
#params.rnn_inner_init_type           = 'normal_radius' 
#params.rnn_inner_init_type           = 'orthogonal'
params.rnn_inner_scale               = 1
params.inner_activation              = 'tanh' #'tanh','sigmoid', 'relu','softplus'
params.rnn_output_scale              = 1 # 1e-6

params.final_layer_type              = 'dense_hingeloss'  # ok with tanh, bad with relu (2-3 rounds) or softplus? 
#params.final_layer_type              = 'sigmoid_hingeloss' # Works with softplus, totally fails with relu
#params.final_layer_type              = 'softmax_binary_crossentropy' #-> seems stuck
params.dense_input_dim               = params.rnn_1_hidden_dim
params.dense_output_dim              = 1

params.dropout                       = 0.75 

# params.optimizer                     = 'SGD'
# params.sgd_lr                        = 1e-5 # 1e-4 by default.
# params.sgd_decay                     = 1e-5
# params.sgd_momentum                  = 0.995
# params.sgd_nesterov                  = True
# params.sgd_clipvalue                   = 10

params.optimizer                    = 'RMSprop'
params.rmsprop_lr                   = 0.001 #0.001 # 0.001 by default.
params.rmsprop_clipvalue            = None

params.do_center_X                   = 1
params.regenerate_datasets           = 0

############## Sanity check on inputs #############

####### Generate dataset #######
if params.final_layer_type   == 'dense_hingeloss':             choice_values = (-1, 1)
elif params.final_layer_type == 'softmax_binary_crossentropy': choice_values = ( 0, 1)
elif params.final_layer_type == 'sigmoid_hingeloss':           choice_values = ( 0, 1)
else: raise Exception('params.final_layer_type not found :: ',params.final_layer_type)
if 'X_train' not in locals() or X_train is None or regenerate_datasets is True or X_train.shape[1] != n_samples:
        print('Generating dataset...')
        (X_train,Y_train,X_test,Y_test) =  generate_dataset()
        Y_test_vals = np.squeeze(Y_test[:,-1,:])
        Y_train_vals = np.squeeze(Y_train[:,-1,:])
else:
    print('Reusing previous datasets.')
#(nb_samples, timesteps, input_dim)
# print('X_train.shape :: ',X_train.shape)
# print('Y_train full :: ',Y_train.shape)
# print('Y_train last :: ',np.squeeze(Y_train[:,-1,:]))
# print('Targeting choice values ',choice_values,'with final layer :: ',params.final_layer_type)

if not set(Y_test_vals) == set(choice_values):
    print('WARNING :: choice_values does not match targets in Y_train')

############### COMPILE MODEL ###########

model = define_and_compile_model(params)
from keras.utils.visualize_util import plot
plot(model, to_file='model.png')
from IPython.display import Image
Image(filename='model.png') 

# ############# Begin training #############
print('Starting training at ',datetime.now())
checkpoint_time = [t[:19] for t in (str(datetime.now()),)]
checkpoint_path = "/home/avaughan/Dropbox/rnn/checkpoint_" + checkpoint_time[0].replace(" ","_").replace(":","-") + ".hdf5"
checkpointer = callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_best_only=True)
earlystopper = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1) # Should take a mode?
nanchecker   = callbacks.NaNChecker(monitor='loss')

model.fit(  X_train, Y_train_vals, 
          nb_epoch=300, batch_size=128, 
          shuffle = 1, verbose=1, validation_data=(X_test, Y_test_vals), 
          callbacks=[checkpointer,earlystopper,nanchecker] )

print('\nDone with cell at',datetime.now())


Generating dataset...
Datasets generated in 15.3 seconds
Centering datasets...
Compiling with parameters as follows:
-------------------  -----------------
dense_input_dim      256
dense_output_dim     1
do_center_X          1
dropout              0.75
final_layer_type     'dense_hingeloss'
inner_activation     'tanh'
optimizer            'RMSprop'
regenerate_datasets  0
rmsprop_clipvalue    None
rmsprop_lr           0.001
rnn_1_hidden_dim     256
rnn_1_input_dim      2
rnn_init_scale       1
rnn_inner_init_type  'identity'
rnn_inner_scale      1
rnn_output_scale     1
-------------------  -----------------
Building  model...
Compiling
...done compiling in 2.6 seconds 2016-01-22 20:40:30.933021
Starting training at  2016-01-22 20:40:30.988577
Train on 100000 samples, validate on 10000 samples
Epoch 1/300
100000/100000 [==============================] - 146s - loss: 0.8420 - val_loss: 0.7772
Epoch 00000: val_loss improved from inf to 0.77720, saving model to /home/avaughan/Dropbox/rnn/c

In [1]:
##### ANALYZE RESPONSES #####
from scipy.optimize import curve_fit

n_trials_validate = 1000
click_bias_validate  = np.random.uniform( 0, 1, (n_trials_validate) )
(X_validate,Y_validate)   = generate_poisson_dataset.generate_click_dataset( 
    n_trials_validate,  click_bias_validate,  n_samples,
    sum_rate=sum_rate, stim_duration=stim_duration,  wait_time=wait_time,  
    sampling_freq=sampling_freq, centered=False,choice_values=choice_values)
Y_validate_vals = np.squeeze(Y_validate[:,-1,:])
left_trials  = Y_validate[:,-1,0] == choice_values[0]
right_trials = Y_validate[:,-1,0] == choice_values[1]
print('\n Parameters of validation set :: ')
print ('mean( left clicks  | right trial ) :: %.0f ' % ( np.mean(np.sum(X_validate[ right_trials , :(sampling_freq*stim_duration),0], axis=1)) ))
print ('mean( right clicks | right trial ) :: %.0f ' % ( np.mean(np.sum(X_validate[ right_trials , :(sampling_freq*stim_duration),1], axis=1)) ))
print ('')
print ('N left choices  :: %.0f ' % (sum(Y_train[:,-1,0] == -1)))
print ('mean( left clicks  | left trial  ) :: %.0f ' % ( np.mean(np.sum(X_validate[ left_trials , :(sampling_freq*stim_duration),0], axis=1)) ))
print ('mean( right cliks  | left trial  ) :: %.0f ' % ( np.mean(np.sum(X_validate[ left_trials , :(sampling_freq*stim_duration),1], axis=1)) ))
print ('')
print ('    min :: %.2f' % (min(X_validate.ravel())))
print ('    max :: %.2f' % (max(X_validate.ravel())))
print ('    mean :: %.2f' % (np.mean(X_validate.ravel())))
print ('\n')

n = 100

#model.fit(X_train, Y_train[:,-1,0], nb_epoch=20, batch_size=5) # Last sample
#model.fit(X_train, Y_train, nb_epoch=10, batch_size=100, validation_split=0.1)
objective_score = model.evaluate(X_validate, Y_validate_vals, batch_size=100)
print('Objective score (',params.final_layer_type,') :: ',objective_score)

validation_prediction = model.predict(X_validate)

if params.final_layer_type   == 'dense_hingeloss':             round_fn = lambda val: (val > 0) * 2 - 1
elif params.final_layer_type == 'softmax_binary_crossentropy': round_fn = lambda val: round(val)
elif params.final_layer_type == 'sigmoid_hingeloss':           round_fn = lambda val: round(val)
else: raise Exception('params.final_layer_type not found :: ', params.final_layer_type)

validation_prediction_round = round_fn(validation_prediction)

# print a.shape
# for i in range(a.shape[0]):
#     plt.plot(a[i,])
#     plt.ylim( (0,1) )
print ('Shape of y_test :: ',         Y_validate[:10,-1,:].shape)
print ('Uniques in y_test :: ',       np.unique(Y_validate[:10,-1,:]))
print ('Shape of test_prediction :: ',validation_prediction.shape)
#print ('Uniques in test_prediction       :: ', np.unique(validation_prediction))
#print ('Uniques in test_prediction_round :: ', np.unique(validation_prediction_round))
uniques = np.unique(validation_prediction_round)
print('Sum of test_prediction_round = uniques[0] :: ',uniques[0],' :: ',sum(validation_prediction_round == uniques[0]))
print('Sum of test_prediction_round = uniques[1] :: ',uniques[1],' :: ',sum(validation_prediction_round == uniques[1]))


# Pull responses and run t-test
def pred_score(target):
    return validation_prediction_round[np.where(Y_validate[:n,-1,0] == target)]
def p_stars(p_value):
    if p < 0.01: return '*'
    return ''



# PLOT RESPONSES
plt.figure()
plt.plot( Y_validate[:n,-1,0] + 0.1 * (np.random.uniform(size=Y_validate[:n,-1,0].shape)-0.5), validation_prediction[:n,0],'.')
plt.violinplot( (pred_score(choice_values[0]) , pred_score(choice_values[1])), positions=choice_values,showmeans=True)
plt.xlim(np.asarray(choice_values) + [-0.5,0.5])

def sigmoid(x, x0, k):
     y = 1 / (1 + np.exp(-k*(x-x0)))
     return y
popt, pcov = curve_fit(sigmoid, click_bias_validate.ravel(), validation_prediction.ravel())
sigmoid_bias       = np.linspace(0,1, 50)
sigmoid_prediction = sigmoid(sigmoid_bias, *popt)
plt.figure()
plt.plot(click_bias_validate.ravel(),validation_prediction + 0.1 * np.random.uniform(size=validation_prediction.shape),'.')
plt.plot(sigmoid_bias,sigmoid_prediction)
plt.xlabel('Nominal click rate')
plt.ylabel('Prediction (raw)')

# MEAN RESPONSE VALUES
print('Choice ',choice_values[0],' :: ',np.mean(pred_score(choice_values[0])),'+/-',  np.var(pred_score(choice_values[0])))
print('Choice ',choice_values[1],' :: ',np.mean(pred_score(choice_values[1])),'+/-', np.var(pred_score(choice_values[1])))

# STATS
[_,p] = sp.stats.ttest_ind(pred_score(choice_values[0]),pred_score(choice_values[1]))
print('p(distributions of response variables are identical) == ',p[0],' ',p_stars(p))

# Crosstab
#pandas.crosstab( np.ravel(validation_prediction_round), np.ravel(Y_validate[:n,-1,0]) ,rownames=['Prediction'],colnames=['Actual'])


NameError: name 'np' is not defined

In [ ]:
############ SANITY CHECK / DEBUG DATSET  #####################

# Generate a dataset of poisson spikes.  Basically just calls generate_click_dataset.
# X_train is set up as n_samples, n_dimensions=2, n_timepoints
# Y_Train is set up as n_samples, n_dimensions=1, n_timepoints
# Cost function will have to match Y_train (ie., only match the end)

n_trials_train = 20000
n_trials_test  = 2000
stim_duration  = 1
wait_time      = 0
sampling_freq  = 1000
sum_rate       = 100
n_samples = (stim_duration+wait_time)*sampling_freq + 1
choice_values = (-1,1) # 

#click_bias_train = np.random.normal(0.5,0.2,(n_trials_train))
#click_bias_test = np.random.normal(0.5,0.2,(n_trials_test))
#click_bias_train[click_bias_train <= 0.1] = 0.1
#click_bias_test[click_bias_test <= 0.1]   = 0.1

click_bias_train = np.random.uniform( 0, 1, (n_trials_train))
click_bias_test  = np.random.uniform( 0, 1, (n_trials_test) )

t_start = time.time()
(X_train,Y_train) = generate_poisson_dataset.generate_click_dataset( n_trials_train, 
                                           click_bias_train, 
                                           n_samples,
                                           sum_rate=sum_rate, 
                                           stim_duration=stim_duration, 
                                           wait_time=wait_time, 
                                           sampling_freq=sampling_freq, 
                                           centered=False)
(X_test,Y_test)   = generate_poisson_dataset.generate_click_dataset( n_trials_test,  
                                           click_bias_test,
                                           n_samples,
                                           sum_rate=sum_rate, 
                                           stim_duration=stim_duration, 
                                           wait_time=wait_time, 
                                           sampling_freq=sampling_freq, 
                                           centered=False)

# Center
#X_train = X_train - np.mean(X_train.ravel())
#X_test  = X_test - np.mean(X_test.ravel())

print('Datasets generated in %.1f seconds' %( time.time() - t_start))
print('X_train.shape :: %s' %(str(X_train.shape)))
print('Y_train.shape :: %s' %(str(Y_train.shape)))
print('X_test.shape  :: %s' %(str(X_test.shape)))
print('Y_test.shape  :: %s' %(str(Y_test.shape)))
print(''     )
print('N right choices :: %.0f ' % (sum(Y_train[:,-1,0] == 1)))

left_trials  = Y_test[:,-1,0] == choice_values[0]
right_trials = Y_test[:,-1,0] == choice_values[1]
print ('mean( left clicks  | right trial ) :: %.0f ' % ( np.mean(np.sum(X_test[ right_trials , :(sampling_freq*stim_duration),0], axis=1)) ))
print ('mean( right clicks | right trial ) :: %.0f ' % ( np.mean(np.sum(X_test[ right_trials , :(sampling_freq*stim_duration),1], axis=1)) ))
print ('')
print ('N left choices  :: %.0f ' % (sum(Y_train[:,-1,0] == -1)))
print ('mean( left clicks  | left trial  ) :: %.0f ' % ( np.mean(np.sum(X_test[ left_trials , :(sampling_freq*stim_duration),0], axis=1)) ))
print ('mean( right cliks  | left trial  ) :: %.0f ' % ( np.mean(np.sum(X_test[ left_trials , :(sampling_freq*stim_duration),1], axis=1)) ))
print ('')
print ('    min :: %.2f' % (min(X_test.ravel())))
print ('    max :: %.2f' % (max(X_test.ravel())))
print ('    mean :: %.2f' % (np.mean(X_test.ravel())))

plt.subplot(2,2,1)
plt.imshow(X_train[:,:,0],aspect='auto')
plt.ylabel('Left Trials (=0)')

plt.subplot(2,2,3)
plt.imshow(X_train[:,:,1],aspect='auto')
plt.ylabel('Right Trials (=1)')

plt.subplot(2,2,2)
plt.plot(X_train[0,:100,0],'.')
plt.ylim((-1.1,1.1))
plt.title('Trial 1 : type %.0f' %(Y_train[0,-1,0]))

plt.subplot(2,2,4)
plt.plot(X_train[0,:100,1],'.')
plt.ylim((-1.1,1.1))

print('\nDone with cell at',datetime.now())


In [ ]:
np.random.uniform()